# 研究者JSONファイルをDBに保存するプログラム

In [ ]:
import configparser
import json
import os
import pickle
import re
import shutil
import time
from glob import glob

import pandas as pd
import requests
from joblib import Parallel, delayed
from sqlalchemy import create_engine
from sqlalchemy.types import Date, Integer, String
from tqdm.notebook import tqdm_notebook

In [ ]:
config = configparser.ConfigParser()
config.read("../../settings/config.ini")

# URL設定
appid = config["CiNii_web_api"]["appid"]

# DB設定
config = configparser.ConfigParser()
config.read("../../settings/config.ini")
username = config["mariadb"]["username"]
password = config["mariadb"]["password"]
url = (
    "mysql+pymysql://"
    + username
    + ":"
    + password
    + "@localhost:3306/"
    + "kaken"
    + "?charset=UTF8MB4"
)
engine = create_engine(url, echo=True)

JSONをパースする関数その1：grantaward_memberから作ったリストに基づいてダウンロードしたJSONファイルをパースする

In [ ]:
def get_researcher(jsonfilepath):
    # ファイルを開く
    with open(jsonfilepath, "r", encoding="utf-8") as f:
        jsondata = json.load(f)
    # ファイル内の件数を取得する
    totalresults = jsondata["totalResults"]
    startindex = jsondata["startIndex"]
    itemsperpage = jsondata["itemsPerPage"]
    if startindex + itemsperpage < totalresults:
        end = itemsperpage
    else:
        end = totalresults - startindex
    # JSONデータをパースする
    researcher_list = []
    researchers = jsondata["researchers"]
    for i in range(end):
        eradcode = researchers[i]["id:person:erad"][0]
        try:
            fullname = researchers[i]["name"]["humanReadableValue"]
        except KeyError:
            fullname = None
        row = [eradcode, fullname]
        researcher_list.append(row)

    dumpfilename = (
        "dump_researcher/"
        + re.search("[0-9]{4}_[0-9]+-[0-9]+.json", jsonfilepath).group()
        + ".dump"
    )
    with open(dumpfilename, "wb") as f:
        pickle.dump(researcher_list, f)

In [ ]:
# dump_researcherフォルダを空にしておく
target_dir = "dump_researcher"
if os.path.isdir(target_dir):
    shutil.rmtree(target_dir)
os.makedirs(target_dir)

In [ ]:
# JSONファイルのリストを作成する
jsonlist = [f for f in glob("json/*.json") if not f.endswith("_1-0.json")]
# Joblibで並列処理する
Parallel(n_jobs=-1, verbose=1)([delayed(get_researcher)(i) for i in jsonlist])

In [ ]:
# リストを結合する
researcher_list = []
for dump in tqdm_notebook(glob("dump_researcher/*.dump")):
    with open(dump, mode="rb") as f:
        l = pickle.load(f)
        researcher_list += l
# リストをデータフレームに変換する
columns = ["eradcode", "fullname"]
df = pd.DataFrame(researcher_list, columns=columns)
# 課題番号に重複がないことを確認して、インデックスに設定する
assert not df["eradcode"].duplicated().any(), "eradcode is duplicated."
df = df.set_index("eradcode")
df

---

In [ ]:
# grantaward_memberから研究者番号リストを取得する
member = pd.read_sql_query("SELECT DISTINCT(eradcode) FROM grantaward_member", engine)
member = [m for m in member["eradcode"].to_list() if m]
len(member)

In [ ]:
# 研究者JSONから研究者番号リストを取得する
researcher = df.index.to_list()
len(researcher)

In [ ]:
# 積集合
seki = list(set(researcher) & set(member))
len(seki)

In [ ]:
# 和集合
wa = list(set(researcher + member))
len(wa)

In [ ]:
# researcherにあってmemberにない研究者番号
r_m = list(set(researcher) - set(member))
len(r_m)

In [ ]:
# memberにあってresearcherにない研究者番号
m_r = list(set(member) - set(researcher))
len(m_r)

---

## 上記researcherで取得できなかった1522件のJSONファイルをダウンロードする

In [ ]:
# JSONファイルをダウンロードする
os.makedirs("json_additional", exist_ok=True)
for i in tqdm_notebook(m_r):
    ROOT_URL = "https://nrid.nii.ac.jp/opensearch/?appid=" + appid
    url = ROOT_URL + "&format=json&qm=" + i
    r = requests.get(url)
    j = r.text
    filename = "json_additional/" + i + ".json"
    with open(filename, mode="w", encoding="utf-8") as f:
        f.write(j)
    time.sleep(5)

In [ ]:
# dump_researcher_additionalフォルダを空にしておく
target_dir = "dump_researcher_additional"
if os.path.isdir(target_dir):
    shutil.rmtree(target_dir)
os.makedirs(target_dir)

In [ ]:
# JSONファイルをリストに変換する
for jsonfilepath in tqdm_notebook(glob("json_additional/*.json")):
    # JSONファイルを開く
    with open(jsonfilepath, "r", encoding="utf-8") as f:
        jsondata = json.load(f)
        # JSONファイル内の件数が1ならパースする
        if jsondata["totalResults"] == 1:
            researchers = jsondata["researchers"]
            eradcode = researchers[0]["id:person:erad"][0]
            try:
                fullname = researchers[0]["name"]["humanReadableValue"]
            except KeyError:
                fullname = None
            row = [eradcode, fullname]
            dumpfilename = (
                "dump_researcher_additional/"
                + re.search("[0-9]{8}.json", jsonfilepath).group()
                + ".dump"
            )
            with open(dumpfilename, "wb") as f:
                pickle.dump(row, f)

In [ ]:
# リストを結合する
researcher_list = []
for dump in tqdm_notebook(glob("dump_researcher_additional/*.dump")):
    with open(dump, mode="rb") as f:
        l = pickle.load(f)
        researcher_list.append(l)
# リストをデータフレームに変換する
columns = ["eradcode", "fullname"]
additional = pd.DataFrame(researcher_list, columns=columns)
# 課題番号に重複がないことを確認して、インデックスに設定する
assert not additional["eradcode"].duplicated().any(), "eradcode is duplicated."
additional = additional.set_index("eradcode")
additional

In [ ]:
# 2つのデータフレームを結合する
df = pd.concat([df, additional])
df

---

In [ ]:
# explodeメソッドでfullname列に入れ子になっている辞書を展開する
fullname = df.explode("fullname")["fullname"].apply(pd.Series)
fullname = fullname.reset_index()
# 縦持ちのデータを横持ちに変換する
fullname = fullname.groupby(["eradcode", "lang"]).apply(lambda d: d["text"].reset_index(drop=True)).unstack()
fullname = fullname.reset_index()
# 列に名前を付ける
fullname.columns=["eradcode", "en", "ja", "kana"]
fullname

In [ ]:
# データベースに保存する
fullname.to_sql("researchers", engine, if_exists="replace")

おしまい